---

# Part 2: Preprocessing Data and Feature Selection

---

## Notebook Summary

This notebooks reads in the cleaned data and goes through some preprocessing of the categorical variables for the purposes of generating an iterative model. Some of the data undergo feature engineering to search for interaction features. The dataset will be train/test split and then scaled for comparing feature variable before running preliminary models and iterating on the most appropriate features. Included in this notebook, the reader will find:

* Preprocessing Data
* Model Baseline
* Polynomial Features
* Ridge Regularization
* Lasson Regularization
* Feature Selection
* Notebook Conclusion

---

## Preprocessing Data

I will begin by dummifying all the categorical variables, so I have the option of running a very overfit model to start. On this model, my plan is to run a ridge and lasso regularization to determine which features might have the most significant impact on the sale price in a less complex model. I will then iterate through different versions of this less complex models to build a final production model and make it interpretable for the purposes of our problem statement.

First, I will import the necessary libraries and then read in the numerically cleaned dataset.

In [1]:
# import requisite libraries and modules for linear regression and feature engineering
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

In [2]:
# read in cleaned dataset from notebook 1 and print out head
homes_train = pd.read_csv('../datasets/train_cleaned.csv')

homes_train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,0.0,13517,Pave,None,IR1,Lvl,...,0,0,None,None,None,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,None,IR1,Lvl,...,0,0,None,None,None,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,None,Reg,Lvl,...,0,0,None,None,None,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,None,Reg,Lvl,...,0,0,None,None,None,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,None,IR1,Lvl,...,0,0,None,None,None,0,3,2010,WD,138500


First, I will go back to those categorical columns misclassified as integers and convert them to strings. Then, I will collect all the string data type columns and dummify the variables all at once.

In [3]:
# reclassify data types identified from notebook 1 from numeric to categorical
homes_train['MS SubClass'] = homes_train['MS SubClass'].astype(str)
homes_train['Mo Sold'] = homes_train['Mo Sold'].astype(str)

homes_train.dtypes

Id                int64
PID               int64
MS SubClass      object
MS Zoning        object
Lot Frontage    float64
                 ...   
Misc Val          int64
Mo Sold          object
Yr Sold           int64
Sale Type        object
SalePrice         int64
Length: 81, dtype: object

In [4]:
# make a list of object data type columns
cols_objs = homes_train.select_dtypes('object').columns

cols_objs

Index(['MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Lot Shape',
       'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood',
       'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond',
       'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating',
       'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature',
       'Mo Sold', 'Sale Type'],
      dtype='object')

In [5]:
# dummify all categorical columns and drop first.
# print the new dataset to check the shape and head
homes_train = pd.get_dummies(data = homes_train, columns = cols_objs, drop_first = True)

print(homes_train.shape)

homes_train.head()

(2051, 288)


,Id,PID,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,...,Mo Sold_8,Mo Sold_9,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_WD
0,109,533352170,0.0,13517,6,8,1976,2005,289.0,533.0,...,0,0,0,0,0,0,0,0,0,1
1,544,531379050,43.0,11492,7,5,1996,1997,132.0,637.0,...,0,0,0,0,0,0,0,0,0,1
2,153,535304180,68.0,7922,5,7,1953,2007,0.0,731.0,...,0,0,0,0,0,0,0,0,0,1
3,318,916386060,73.0,9802,5,5,2006,2007,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,255,906425045,82.0,14235,6,8,1900,1993,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


Our categorical variables have all been dummified, and we now have a dataframe with 288 columns instead of 81.

---

## Baseline Model

With the data preprocessed, I will begin by creating a baseline model. The baseline model will include all of the features, both numerical and categorical after standard scaling. The baseline model will compare predictions to the mean of the sale price as our target variable. This baseline model like all of the linear regression models which follow will be evaluated based on the Pearson coefficient of determination, the R<sup>2</sup> score.

In [6]:
# choose features and target for baseline model and do a train/test split
X_null = homes_train.drop(columns = ['Id', 'PID', 'SalePrice'])
y_null = homes_train['SalePrice']

X_train_null, X_test_null, y_train_null, y_test_null = train_test_split(X_null, y_null, random_state = 42)

In [7]:
# Creare a list of y_train mean to compare against the y_test set and find the R2 score
base_nulls = [y_train_null.mean()] * len(y_test_null)

r2_score(y_test_null, base_nulls)

-0.00043273813883448753

Already in our baseline model, I can see that just based on the mean of the sale price in the training set, we have an abysmally low negative coefficient of determination at a value of -0.00143. This means that any positive R<sup>2</sup> value is already an improvement over the baseline model, but of course we will attempt to fit our model so the evaluation metric is increasingly closer to one.

---

## Polynomial Features

With all our data preprocessed and a baseline model established, I will create a train/test split in our training data and begin with creating some polynomial features in our dataset and then standard scaling. The goal here is to begin to scan for any interaction features and then, slowly whittle down to some of the most salient features to make a more interpretable production model.

In [8]:
# choose features and target for polynomial model
X = homes_train.drop(columns = ['Id', 'PID', 'SalePrice'])
y = homes_train['SalePrice']

In [9]:
# create a Pipeline which transforms the dataset with polynomial features
# transforms all features using a standard scaler
# instantiates a linear regression
poly_pipe = Pipeline([
        ('poly', PolynomialFeatures(include_bias = False)),
        ('sc', StandardScaler()),
        ('lr', LinearRegression())
    ])
    
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
    
poly_pipe.fit(X_train, y_train)

Pipeline(steps=[('poly', PolynomialFeatures(include_bias=False)),
                ('sc', StandardScaler()), ('lr', LinearRegression())])

In [10]:
# prints out cross-validation and R2 scores for train and test
print(f'Cross-Validation Score: {cross_val_score(poly_pipe, X_train, y_train).mean()}')
print(f'Cross-Validation Scores: {cross_val_score(poly_pipe, X_train, y_train)}')
print(f'Training Score: {poly_pipe.score(X_train, y_train)}')
print(f'Test Score: {poly_pipe.score(X_test, y_test)}')

Cross-Validation Score: 0.4395038549537733
Cross-Validation Scores: [ 0.84114048 -0.46236391  0.85706834  0.55081694  0.41085743]
Training Score: 1.0
Test Score: 0.8706088732645141


As suspected, this model is extremely overfit. With over tens of thousands of features, the model performs extremely well in the training dataset with an R<sup>2</sup> value of approximately 1. However, the same model in the test data has an R<sup>2</sup> value of approximately 0.87, failing miserably to predict, and performs even more poorly in cross-validation with a score of 0.44.

Next, I want to look more closely at the largest coefficients in magnitude to determine if there are any salient interaction features to try to keep in our final production model. I might also make note of whether specific features keep showing up repeatedly in these high magnitude correlation coefficients to keep them in the final production model as well.

In [11]:
# saves the coefficients as a series
poly_coefs = pd.Series(poly_pipe['lr'].coef_, index = poly_pipe[:-1].get_feature_names_out()).sort_values(ascending = False)

poly_coefs

Bsmt Exposure_Gd Mo Sold_3               1222.222964
Mas Vnr Type_Stone Garage Finish_RFn     1046.168563
Bsmt Full Bath Fireplace Qu_TA           1012.215316
Screen Porch Mas Vnr Type_Stone          1002.016080
Neighborhood_NridgHt Bsmt Exposure_Gd     970.170756
                                            ...     
Bldg Type_TwnhsE BsmtFin Type 1_GLQ     -1183.277892
Neighborhood_NridgHt Exter Qual_Gd      -1185.228927
Neighborhood_NridgHt Bsmt Qual_Gd       -1296.601814
Mas Vnr Type_Stone BsmtFin Type 1_GLQ   -1430.856192
Lot Frontage Bldg Type_TwnhsE           -1493.835827
Length: 41040, dtype: float64

In [12]:
# sorts 10 largest magnitude positive coefficients
poly_coefs.sort_values(ascending = False).head(10)

Bsmt Exposure_Gd Mo Sold_3                1222.222964
Mas Vnr Type_Stone Garage Finish_RFn      1046.168563
Bsmt Full Bath Fireplace Qu_TA            1012.215316
Screen Porch Mas Vnr Type_Stone           1002.016080
Neighborhood_NridgHt Bsmt Exposure_Gd      970.170756
Lot Frontage Mas Vnr Type_Stone            943.470048
Neighborhood_CollgCr Garage Finish_RFn     932.916922
Neighborhood_NridgHt Fence_MnPrv           920.476377
Screen Porch Garage Type_BuiltIn           918.634791
Mas Vnr Area Mas Vnr Type_Stone            897.012154
dtype: float64

In [13]:
# sorts 10 largest magnitude negative coefficients
poly_coefs.sort_values().head(10)

Lot Frontage Bldg Type_TwnhsE             -1493.835827
Mas Vnr Type_Stone BsmtFin Type 1_GLQ     -1430.856192
Neighborhood_NridgHt Bsmt Qual_Gd         -1296.601814
Neighborhood_NridgHt Exter Qual_Gd        -1185.228927
Bldg Type_TwnhsE BsmtFin Type 1_GLQ       -1183.277892
Mas Vnr Type_Stone Exter Qual_Gd          -1114.658407
Neighborhood_NridgHt Mo Sold_8            -1077.187424
Neighborhood_Somerst Sale Type_New        -1073.893851
Neighborhood_NridgHt BsmtFin Type 1_GLQ   -1070.990011
Neighborhood_NoRidge Heating QC_Gd        -1017.593436
dtype: float64

Looking at the interaction features and some of the correlation coefficients associated with them, I plan on using some common patterns to see which features might be useful in fitting my final production model later, even if they are not directly related to my problem statement. I notice that there are a handful of neighborhoods which are either positively or negatively correlated with the sales price across multiple interaction features, including Northridge, Northridge Heights, and College Creek. There also appear to be strong correlations with home sales during March and July. Basement finish type of good living quarters and mason veneer type stone also appear to have a strong correlation. Regarding my problem statement of square footage of home features, only the basement finish square footage and wood deck square footage show up albeit without any discrernible pattern.

I will now return to the original features and dummified categorical variables. First, I will just run a multi-linear regression model to establish how overfit the model is with all the features included. After, I will apply Ridge and LASSO regularization methods to avoid overfitting and compare to the original regression model. Additionally with the LASSO method, I intend to zero out as many correlation coefficients as possible to come up with a more simplified, well-fit model and hopefully corroborate some of the interaction features that I am seeing as patterns above.

In [14]:
# creates a pipline to scale features and fit a linear regression
ss_pipe = Pipeline([
        ('sc', StandardScaler()),
        ('lr', LinearRegression())
    ])
    
ss_pipe.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()), ('lr', LinearRegression())])

In [15]:
# print cross-validation and R2 scores for multi-linear regression with all dataset features  
print(f'Cross-Validation Score: {cross_val_score(ss_pipe, X_train, y_train).mean()}')
print(f'Cross-Validation Score: {cross_val_score(ss_pipe, X_train, y_train)}')
print(f'Training Score: {ss_pipe.score(X_train, y_train)}')
print(f'Test Score: {ss_pipe.score(X_test, y_test)}')

Cross-Validation Score: -2.5654403002891895e+20
Cross-Validation Score: [-7.32211246e+16 -8.47907477e+16 -1.22837665e+21 -1.58663518e+18
 -5.25988493e+19]
Training Score: 0.9468961977986194
Test Score: -3.415939454822445e+21


As we can see on the training score of approximately 0.95 in comparison to both the cross-validation score and test score with deeply negative values, this model is severely overfit, far more overfit than even my polynomial features model. I will now use a Ridge regularlization method on this linear regression to correct the overfit.

---

## Ridge Regularization

The following Ridge regularization works off my overfit multi-linear regression model above. The goal is to begin to get a better fit model and determine which variables can be eliminated from examination altogether when drafting an initial preliminary model.

In [16]:
# creates a pipline to scale features and fit a Ridge regularization
ridge_pipe = Pipeline([
        ('sc', StandardScaler()),
        ('ridge', Ridge())
    ])

ridge_pipe.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()), ('ridge', Ridge())])

In [17]:
# print cross-validation and R2 scores for Ridge regularization with all dataset features  
print(f'Cross-Validation Score: {cross_val_score(ridge_pipe, X_train, y_train).mean()}')
print(f'Cross-Validation Scores: {cross_val_score(ridge_pipe, X_train, y_train)}')
print(f'Ridge Training Score: {ridge_pipe.score(X_train, y_train)}')
print(f'Ridge Test Score: {ridge_pipe.score(X_test, y_test)}')

Cross-Validation Score: 0.6982081012488933
Cross-Validation Scores: [0.92351214 0.64698993 0.59686821 0.78978617 0.53388406]
Ridge Training Score: 0.9461734026859759
Ridge Test Score: 0.9146628419162716


Already just using a Ridge alpha of 1.0 as the default, we can see that the test score has improved dramatically from a negative number to approximately 0.91. The default alpha value has already created a model with much better predictive power than the previous scaled model of polynomial features. The cross-validation score though is stil lower at about 0.70, far removed from the training score of 0.95, so this model still has a high degree of variance. That means that this is likely not an ideal value for alpha. I will now use the Ridge cross-validation method to check multiple values of alpha to optimize the model.

In [18]:
# creates an array of 100 alpha values to test, scales features, and then runs cross-validation for optimal alpha
alphas = np.logspace(0, 5, 100)

ridge_cv_pipe = Pipeline([
        ('sc', StandardScaler()),
        ('ridge_cv', RidgeCV(alphas = alphas, cv = 5))
    ])

ridge_cv_pipe.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('ridge_cv',
                 RidgeCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.19926714e+00, 3.59381366e+00,
       4.03701726e+00, 4.53487851e+00, 5.09413801e+00, 5.72236766e+00,
       6.42807312e+00, 7.22080902e+00, 8.1113...
       6.89261210e+03, 7.74263683e+03, 8.69749003e+03, 9.77009957e+03,
       1.09749877e+04, 1.23284674e+04, 1.38488637e+04, 1.55567614e+04,
       1.74752840e+04, 1.96304065e+04, 2.20513074e+04, 2.47707636e+04,
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05]),
                         cv=5))])

In [19]:
# prints optimal alpha and cross-validation score
print(f"Ridge Optimal Alpha: {ridge_cv_pipe['ridge_cv'].alpha_}")
print(f"Ridge Best Score: {ridge_cv_pipe['ridge_cv'].best_score_}")

Ridge Optimal Alpha: 599.4842503189409
Ridge Best Score: 0.8389597241618055


In [20]:
# print optimal alpha training and test scores
print(f'Ridge Optimal Alpha Training Score: {ridge_cv_pipe.score(X_train, y_train)}')
print(f'Ridge Optimal Alpha Test Score: {ridge_cv_pipe.score(X_test, y_test)}')

Ridge Optimal Alpha Training Score: 0.9038950228984197
Ridge Optimal Alpha Test Score: 0.8897218904154145


First, this optimal alpha vaue of approximately 599, has a best cross-validation score of ~0.84 with training and test scores at 0.90 and 0.89, respectively. This means that with the Ridge regularization, the model is a much better fit. It is still slightly overfit since the cross-validation score is somewhat removed our training score.

Now, I will look at the strong positive and negative correlation coefficients to establish any patterns for model tuning.

In [21]:
# saves the coefficients as a series
ridge_cv_coefs = pd.Series(ridge_cv_pipe['ridge_cv'].coef_, 
                           index = ridge_cv_pipe[:-1]\
                           .get_feature_names_out()
                          ).sort_values(ascending = False)

ridge_cv_coefs

Overall Qual            8499.681380
Gr Liv Area             6544.143495
Neighborhood_NridgHt    6530.237833
Neighborhood_StoneBr    5317.875231
1st Flr SF              4843.047085
                           ...     
Kitchen Qual_Gd        -3277.845387
Bsmt Qual_Gd           -3601.363348
Exter Qual_TA          -4030.414578
Kitchen Qual_TA        -4070.545863
Misc Val               -5569.128251
Length: 285, dtype: float64

In [22]:
# sorts 5 largest magnitude positive coefficients
ridge_cv_coefs.sort_values(ascending = False).head(5)

Overall Qual            8499.681380
Gr Liv Area             6544.143495
Neighborhood_NridgHt    6530.237833
Neighborhood_StoneBr    5317.875231
1st Flr SF              4843.047085
dtype: float64

In [23]:
# sorts 5 largest magnitude negative coefficients
ridge_cv_coefs.sort_values().head(5)

Misc Val          -5569.128251
Kitchen Qual_TA   -4070.545863
Exter Qual_TA     -4030.414578
Bsmt Qual_Gd      -3601.363348
Kitchen Qual_Gd   -3277.845387
dtype: float64

In [24]:
# sorts coefficients equal to zero after Ridge
ridge_cv_coefs[ridge_cv_coefs == 0]

Utilities_NoSewr     0.0
Condition 2_RRAe     0.0
MS Zoning_I (all)    0.0
Utilities_NoSeWa     0.0
dtype: float64

Upon inspecting the correlation coefficients, I am noticing some common patterns from my exploratory data analysis. First, a lot of the same numerical features from my EDA have strong correlation coefficients with the sale price, namely overal quality, above ground living area, 1st floor square footage, masonry veneer area, and total basement square footage. I can also see that some of the neighorhoods noted in the polynomials features with a recurring patter, including Northridge Heights and Northridge show up with strong positive correlation coefficients. Regarding negative correlations, virtually all of the strongest negative correlations have to do with typical or average quality in the condition of parts of the home, including kitchen, basement, and exterior. Lastly, the Ridge regularization has selected out only four features listed above.

Next, I will run a LASSO regularization model to see if it is a better fit than the Ridge regularalization and if possible, I will also eliminate as many coefficients as possible.

---

## LASSO Regularization

In this section, I will begin once again with the same features from the overfit multi-linear regression model and fit and cross-validate a LASSO regularization. By the end of this section, I should have generated a list of features which zero out after cross-validation and drop them from my preliminary model for tuning in the next notebook.

In [25]:
# import warnings to ignore warning in LASSO Pipeline
import warnings

warnings.filterwarnings('ignore')

In [26]:
# creates a pipline to scale features and fit a LASSO regularization
lasso_pipe = Pipeline([
        ('sc', StandardScaler()),
        ('lasso', Lasso())
    ])

lasso_pipe.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()), ('lasso', Lasso())])

In [27]:
# print cross-validation and R2 scores for LASSO regularization with all dataset features 
print(f'Cross-Validation Score: {cross_val_score(lasso_pipe, X_train, y_train).mean()}')
print(f'Cross-Validation Scores: {cross_val_score(lasso_pipe, X_train, y_train)}')
print(f'LASSO Training Score: {lasso_pipe.score(X_train, y_train)}')
print(f'LASSO Test Score: {lasso_pipe.score(X_test, y_test)}')

Cross-Validation Score: 0.6195633595323418
Cross-Validation Scores: [0.92508079 0.58180246 0.29153595 0.78287194 0.51652566]
LASSO Training Score: 0.9467966740087187
LASSO Test Score: 0.9124630525214338


In this first run of this LASSO regluarization with a default alpha value of 1.0, I can already see a better fit than the multi-linear regression model. Also, the LASSO regularization is performing about the same degree of overfit as the original Ridge regluarization with the same default alpha value, both having high training and test scores but lower performing cross-validation scores.

Just as with the Ridge regularization, I will now be using the LASSO cross-validation to find the optimal alpha value.

In [28]:
# creates an array of 100 alpha values to test, scales features, and then runs cross-validation for optimal alpha
l_alphas = np.logspace(0, 5, 100)

lasso_cv_pipe = Pipeline([
        ('sc', StandardScaler()),
        ('lasso_cv', LassoCV(alphas = l_alphas))
    ])

lasso_cv_pipe.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('lasso_cv',
                 LassoCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.19926714e+00, 3.59381366e+00,
       4.03701726e+00, 4.53487851e+00, 5.09413801e+00, 5.72236766e+00,
       6.42807312e+00, 7.22080902e+00, 8.1113...
       6.89261210e+03, 7.74263683e+03, 8.69749003e+03, 9.77009957e+03,
       1.09749877e+04, 1.23284674e+04, 1.38488637e+04, 1.55567614e+04,
       1.74752840e+04, 1.96304065e+04, 2.20513074e+04, 2.47707636e+04,
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05])))])

In [29]:
# print optimal alpha training and test scores
print(f"LASSO Optimal Alpha: {lasso_cv_pipe['lasso_cv'].alpha_}")
print(f'LASSO Optimal Alpha Training Score: {lasso_cv_pipe.score(X_train, y_train)}')
print(f'LASSO Optimal Alpha Test Score: {lasso_cv_pipe.score(X_train, y_train)}')

LASSO Optimal Alpha: 673.4150657750821
LASSO Optimal Alpha Training Score: 0.909976944308284
LASSO Optimal Alpha Test Score: 0.909976944308284


In [30]:
# saves the coefficients as a series
lasso_cv_coefs = pd.Series(lasso_cv_pipe['lasso_cv'].coef_, 
                           index = lasso_cv_pipe[:-1]\
                           .get_feature_names_out()
                          ).sort_values(ascending = False)

lasso_cv_coefs

Gr Liv Area             18381.475386
Overall Qual            14927.034292
Neighborhood_NridgHt     8799.617145
Year Built               6847.324612
Neighborhood_StoneBr     6700.939812
                            ...     
Pool QC_Gd              -5243.874555
Exter Qual_TA           -5500.408802
Kitchen Qual_Gd         -7002.361382
Kitchen Qual_TA         -7980.283107
Misc Val                -8755.355828
Length: 285, dtype: float64

In [31]:
# sorts 5 largest magnitude positive coefficients
lasso_cv_coefs.sort_values(ascending = False).head(5)

Gr Liv Area             18381.475386
Overall Qual            14927.034292
Neighborhood_NridgHt     8799.617145
Year Built               6847.324612
Neighborhood_StoneBr     6700.939812
dtype: float64

In [32]:
# sorts 5 largest magnitude negative coefficients
lasso_cv_coefs.sort_values().head(5)

Misc Val          -8755.355828
Kitchen Qual_TA   -7980.283107
Kitchen Qual_Gd   -7002.361382
Exter Qual_TA     -5500.408802
Pool QC_Gd        -5243.874555
dtype: float64

In [33]:
# stores zeroed out coefficient from LASSO cv
lasso_cv_zeros = lasso_cv_coefs[lasso_cv_coefs == 0]

lasso_cv_zeros

BsmtFin Type 1_None    -0.0
BsmtFin Type 2_Rec     -0.0
BsmtFin Type 1_LwQ     -0.0
BsmtFin Type 2_None    -0.0
BsmtFin Type 1_BLQ      0.0
                       ... 
Condition 1_RRNe       -0.0
Exterior 2nd_Brk Cmn    0.0
Exterior 2nd_AsphShn    0.0
Condition 1_RRNn       -0.0
House Style_1Story      0.0
Length: 168, dtype: float64

With an optimal alpha value of approximately 673, this LASSO regularization is now outperforming the Ridge regularization with both training and test scores at aproximately 0.91. Also, while Ridge only eliminated four features, this LASSO cross-validation optimization eliminated 168, whittling down our features by more than half. I will return to this finding in feature selection shortly. Once again, just like the Ridge regularization, many of the same highly positively and negatively correlated variables show up again. Using what we have learned from the Ridge and LASSO regularization, it is time to make some base decisions of feature selection related to the problem statement at hand.

---

## Feature Selection

Before continuing with our model tuning in the next notebook, I will attempt to automate some feature selection. First, since my problem statement is related to which home features' areas should be maximized or minimized for improved home sales prices, I will make sure to include virtually all the measurements of feature area to start. I will use the zeroed out features from Ridge and LASSO to eliminate extraneous features from our model as long as they are not area-related features. Then, I will also use the ten largest magnitude features from both the Ridge and LASSO regularizations to draw up a preliminary, better fit and interpretable model.

In [34]:
# creates a list of all the features which include square footage or area in the column name
sf_features = [col for col in homes_train.columns if ('SF' in col) or ('Area' in col)]

sf_features

['Lot Area',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Pool Area',
 'House Style_SFoyer']

In [35]:
# removes the House Style_SFoyer from the list since it is not a square footage feature
sf_features.pop()

sf_features

['Lot Area',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Pool Area']

In [36]:
# appends three area features to square footage list which do not have SF in their column name.
area = ['Enclosed Porch', '3Ssn Porch', 'Screen Porch']

for feature in area:
    sf_features.append(feature)
    
sf_features

['Lot Area',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Pool Area',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch']

In [37]:
# create a list of features which are zeroed out in LASSO cross-validation but exludes the square footage list.
drop_features = [col for col in lasso_cv_zeros.index if col not in sf_features]

print(len(drop_features))

161


In [38]:
# drops all the columns which were zeroed out in LASSO
homes_train.drop(columns = drop_features, inplace = True)

homes_train.head()

,Id,PID,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Misc Feature_Gar2,Misc Feature_Othr,Misc Feature_Shed,Mo Sold_10,Mo Sold_5,Mo Sold_7,Mo Sold_8,Sale Type_Con,Sale Type_New,Sale Type_Oth
0,109,533352170,13517,6,8,1976,2005,289.0,533.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,544,531379050,11492,7,5,1996,1997,132.0,637.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,153,535304180,7922,5,7,1953,2007,0.0,731.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,318,916386060,9802,5,5,2006,2007,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,255,906425045,14235,6,8,1900,1993,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


After controlling for all the selected square footage and area features to maintain in our dataset, I removed all 162 features which ended up with a zero correlation coefficient after the LASSO cross-validation. I will now use the largest magnitude coefficients from both the Ridge and the LASSO cross validation models to generate the first model aimed at identifying the key features whole square footage most tightly correlates with home sale price.

In [39]:
#creates a Series with the 10 highest magnitude coefficients from Ridge cross-validation above.
big_ridge = abs(ridge_cv_coefs).sort_values(ascending = False).head(10)

big_ridge

Overall Qual            8499.681380
Gr Liv Area             6544.143495
Neighborhood_NridgHt    6530.237833
Misc Val                5569.128251
Neighborhood_StoneBr    5317.875231
1st Flr SF              4843.047085
Mas Vnr Area            4524.029949
TotRms AbvGrd           4415.408926
Neighborhood_NoRidge    4229.534340
Total Bsmt SF           4210.964463
dtype: float64

In [40]:
#creates a Series with the 10 highest magnitude coefficients from LASSO cross-validation above.
big_lasso = abs(lasso_cv_coefs).sort_values(ascending = False).head(10)

big_lasso

Gr Liv Area             18381.475386
Overall Qual            14927.034292
Neighborhood_NridgHt     8799.617145
Misc Val                 8755.355828
Kitchen Qual_TA          7980.283107
Kitchen Qual_Gd          7002.361382
Year Built               6847.324612
Neighborhood_StoneBr     6700.939812
Exter Qual_TA            5500.408802
Total Bsmt SF            5276.933324
dtype: float64

In [41]:
# create initial list of features for preliminary model
features = list(set(big_ridge.index).union(set(big_lasso.index).union(set(sf_features))))

print(len(features))

features

27


['Exter Qual_TA',
 'Neighborhood_NridgHt',
 'Lot Area',
 'Open Porch SF',
 'Garage Area',
 '2nd Flr SF',
 'BsmtFin SF 2',
 'Neighborhood_NoRidge',
 'Gr Liv Area',
 'Bsmt Unf SF',
 'Pool Area',
 'BsmtFin SF 1',
 'Low Qual Fin SF',
 'Misc Val',
 'Overall Qual',
 'Enclosed Porch',
 'Total Bsmt SF',
 'Kitchen Qual_Gd',
 'Wood Deck SF',
 '1st Flr SF',
 'Neighborhood_StoneBr',
 'TotRms AbvGrd',
 'Kitchen Qual_TA',
 'Mas Vnr Area',
 '3Ssn Porch',
 'Year Built',
 'Screen Porch']

I have now whittled my model features down to 27, which include all the square footage and area features, as well as any of the highest magnitude correlation coefficients from either our Ridge or LASSO models above. I will now fit this first preliminary model and use it to iterate in the next notebook for model tuning and a finalized production model.

In [42]:
# Sets the features and target variable, does a train/test split, scales data, and then fits to linear regression.
X_prelim = homes_train[features]
y_prelim = homes_train['SalePrice']

X_train_pl, X_test_pl, y_train_pl, y_test_pl = train_test_split(X_prelim, y_prelim, random_state = 42)
    
ss_pipe.fit(X_train_pl, y_train_pl)

Pipeline(steps=[('sc', StandardScaler()), ('lr', LinearRegression())])

In [43]:
print(f'Cross-Validation Score: {cross_val_score(ss_pipe, X_train_pl, y_train_pl).mean()}')
print(f'Cross-Validation Score: {cross_val_score(ss_pipe, X_train_pl, y_train_pl)}')
print(f'Training Score: {ss_pipe.score(X_train_pl, y_train_pl)}')
print(f'Test Score: {ss_pipe.score(X_test_pl, y_test_pl)}')

Cross-Validation Score: 0.7973174490897842
Cross-Validation Score: [0.86032016 0.84762428 0.85748197 0.80349784 0.617663  ]
Training Score: 0.8452523357807902
Test Score: 0.8730639392307981


Based on the square footage features and the highest magnitude features from the Ridge LASSO models, this preliminary model manages to achive a pretty successful R<sup>2</sup> of approximately 0.85 on the training and 0.87 on the test. Therefore, this is already a pretty well-fit model for making inferences and predictions about our housing data in Ames, IA. Let's take a quick look at the correlation coefficients in the multi-linear regression model, just to get a glimpse into which square footage features appear to have an outsized impact on the sale price. Then in the next notebook, I will use repeated LASSO regularization to begin to eliminate features and refine towards a final production model.

In [44]:
pd.Series(ss_pipe['lr'].coef_, index = ss_pipe[:-1]\
                           .get_feature_names_out()
                          ).sort_values(ascending = False)

Gr Liv Area             6.220324e+16
Total Bsmt SF           2.449806e+16
Overall Qual            2.456440e+04
Year Built              1.010978e+04
Neighborhood_NridgHt    9.020234e+03
Neighborhood_StoneBr    7.219785e+03
Lot Area                6.486699e+03
Screen Porch            5.989820e+03
Garage Area             5.913997e+03
TotRms AbvGrd           5.649033e+03
Neighborhood_NoRidge    5.613988e+03
Wood Deck SF            3.860186e+03
Mas Vnr Area            2.379791e+03
3Ssn Porch              2.063072e+03
Enclosed Porch          1.769038e+03
Open Porch SF           1.559964e+03
Exter Qual_TA          -3.276835e+03
Pool Area              -4.968268e+03
Misc Val               -7.444213e+03
Kitchen Qual_Gd        -1.232711e+04
Kitchen Qual_TA        -1.549288e+04
Low Qual Fin SF        -6.476340e+15
BsmtFin SF 2           -9.145781e+15
Bsmt Unf SF            -2.386686e+16
BsmtFin SF 1           -2.524061e+16
1st Flr SF             -4.957185e+16
2nd Flr SF             -5.220410e+16
d

Similar to what I saw in my data analysis in notebook 1, the living area and basement area have a large positive correlation with sale price. Surprisingly, total basement, first floor, and second floor square footage all have huge negative correlations. I imagine that this might be due to a combination of multi-colinear variables creating higher variance (total basement and first floor square footage noted in notebook 1). When I iterate on this model in the next notebook, I will try to eliminate some multi-colinear variable and determine if this improves our model fit.

I will finish up by saving two updated versions of the dataset, one with all the dropped features at the start of feature selection and another with just the ID, PID, preliminary model features, and sale price. I am saving the first dataset with just the dropped features in case, I wish to add features when refining my model, but I imagine that I will only be dropping features at this point.

In [45]:
# saves the dataset with just the dropped columns which zeroed out in Ridge and LASSO cv
homes_train.to_csv('../datasets/train_dropped.csv', index = False)

In [46]:
# makes a list of columns to keep with preliminary features, id, and sale price
keep_cols = [col for col in homes_train.columns if ((col in features) or (col == 'Id') or (col == 'SalePrice'))]

print(len(keep_cols))
keep_cols

29


['Id',
 'Lot Area',
 'Overall Qual',
 'Year Built',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'TotRms AbvGrd',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'SalePrice',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_StoneBr',
 'Exter Qual_TA',
 'Kitchen Qual_Gd',
 'Kitchen Qual_TA']

In [47]:
# creates a new dataframe with the columns to keep
homes_prelim = homes_train[keep_cols]

print(homes_prelim.shape)
homes_prelim.head()

(2051, 29)


,Id,Lot Area,Overall Qual,Year Built,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Screen Porch,Pool Area,Misc Val,SalePrice,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_StoneBr,Exter Qual_TA,Kitchen Qual_Gd,Kitchen Qual_TA
0,109,13517,6,1976,289.0,533.0,0.0,192.0,725.0,725,...,0,0,0,130500,0,0,0,0,1,0
1,544,11492,7,1996,132.0,637.0,0.0,276.0,913.0,913,...,0,0,0,220000,0,0,0,0,1,0
2,153,7922,5,1953,0.0,731.0,0.0,326.0,1057.0,1057,...,0,0,0,109000,0,0,0,1,1,0
3,318,9802,5,2006,0.0,0.0,0.0,384.0,384.0,744,...,0,0,0,174000,0,0,0,1,0,1
4,255,14235,6,1900,0.0,0.0,0.0,676.0,676.0,831,...,0,0,0,138500,0,0,0,1,0,1


In [48]:
# saves the dataframe with 27 key features to datasets directory
homes_prelim.to_csv('../datasets/prelim_model.csv', index = False)

---

## Notebook Conclusion

In this notebook, I preprocessed data to expand my investigation to include categorical features in a baseline model which performed poorly in test scoring and was highly overfit. I also applied polynomial features in an attempt to look for any patterns in high interaction features. After fitting and cross-validating both Ridge and LASSO regularizations, I was able to generate a comprehensive list of features for a more automated selection. These features included the measures of home area sizes relevant to my problem statement, as well as features with the highest magnitude correlation coefficients in both regularization model. A preliminary model was built based on these key features to keep and is already scoring well.

In the next notebook, I will be model tuning to see which features can be dropped from our model, and by extension, which home features sizes are less revelant in inferring the value of the home for sale.